In [1]:
import pandas as pd
import numpy as np
# You might use TA-Lib or another library to compute technical indicators

# Assume 'df' is a DataFrame containing your OHLC data with columns: 'Open', 'High', 'Low', 'Close'

import pandas as pd

# Example OHLC data
data = {
    'Open': [100, 105, 102, 107, 110],
    'High': [105, 110, 108, 112, 115],
    'Low': [95, 100, 101, 105, 108],
    'Close': [102, 107, 105, 110, 112]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Now you can use the DataFrame with your functions
ha_data = compute_heikin_ashi(df)
ema200 = compute_ema(df['Close'], period=200)
stochK, stochD = compute_stochastic_rsi(df)


def compute_heikin_ashi(df):
    ha_df = df.copy()
    ha_df['HA_Close'] = (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4
    ha_df['HA_Open'] = 0.0
    ha_df['HA_High'] = 0.0
    ha_df['HA_Low'] = 0.0

    # Initialize the first HA candle
    ha_df.at[0, 'HA_Open'] = (df.at[0, 'Open'] + df.at[0, 'Close']) / 2

    # Calculate subsequent HA_Open values
    for i in range(1, len(df)):
        ha_df.at[i, 'HA_Open'] = (ha_df.at[i-1, 'HA_Open'] + ha_df.at[i-1, 'HA_Close']) / 2

    ha_df['HA_High'] = ha_df[['HA_Open', 'HA_Close', 'High']].max(axis=1)
    ha_df['HA_Low'] = ha_df[['HA_Open', 'HA_Close', 'Low']].min(axis=1)
    return ha_df

def compute_ema(series, period=200):
    # This is a simple EMA calculation; in practice, use TA-Lib or Pandas' ewm
    return series.ewm(span=period, adjust=False).mean()

def compute_stochastic_rsi(df, period=14, smoothK=3, smoothD=3):
    # Compute RSI first (this is one way; many libraries have direct implementations)
    delta = df['Close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    # Then compute Stochastic RSI
    min_rsi = rsi.rolling(window=period).min()
    max_rsi = rsi.rolling(window=period).max()
    stoch_rsi = (rsi - min_rsi) / (max_rsi - min_rsi)
    
    # Smooth the %K line
    stoch_rsi_K = stoch_rsi.rolling(window=smoothK).mean()
    # Smooth the %D line
    stoch_rsi_D = stoch_rsi_K.rolling(window=smoothD).mean()
    
    return stoch_rsi_K, stoch_rsi_D

# Process data
ha_data = compute_heikin_ashi(df)
ema200 = compute_ema(df['Close'], period=200)
stochK, stochD = compute_stochastic_rsi(df)

# Loop over new candles (this would be part of your real-time data feed or backtest loop)
for i in range(1, len(df)):
    current_price = df.at[i, 'Close']
    current_ema = ema200[i]
    
    # Determine market trend based on EMA 200
    if current_price > current_ema:
        position = 'long'
    elif current_price < current_ema:
        position = 'short'
    else:
        position = None  # No clear trend
    
    # Check for entry signals only if a clear trend is detected
    if position == 'long':
        # Look for a bullish crossover in the oversold region in Stochastic RSI
        # (You’ll need to define your oversold threshold, e.g., below 0.2)
        if stochK[i-1] < 0.2 and stochK[i] > stochD[i]:
            # Check the candle pattern: current HA candle's body is longer and no lower shadow
            prev_body = abs(ha_data.at[i-1, 'HA_Close'] - ha_data.at[i-1, 'HA_Open'])
            current_body = abs(ha_data.at[i, 'HA_Close'] - ha_data.at[i, 'HA_Open'])
            lower_shadow = ha_data.at[i, 'HA_Open'] - ha_data.at[i, 'HA_Low']
            if current_body > prev_body and lower_shadow < some_small_threshold:
                # Trigger long entry
                entry_price = current_price
                stop_loss = entry_price * (1 - 0.015)  # 1.5% stop loss
                take_profit = entry_price * (1 + 0.03)  # 3% take profit
                print("Long Entry at", entry_price)
                # Execute trade logic here

    elif position == 'short':
        # For shorts, you might check for a bearish crossover in the overbought region (e.g., above 0.8)
        if stochK[i-1] > 0.8 and stochK[i] < stochD[i]:
            # Check for a candle pattern with a longer body and no upper shadow
            prev_body = abs(ha_data.at[i-1, 'HA_Close'] - ha_data.at[i-1, 'HA_Open'])
            current_body = abs(ha_data.at[i, 'HA_Close'] - ha_data.at[i, 'HA_Open'])
            upper_shadow = ha_data.at[i, 'HA_High'] - ha_data.at[i, 'HA_Open']
            if current_body > prev_body and upper_shadow < some_small_threshold:
                # Trigger short entry
                entry_price = current_price
                stop_loss = entry_price * (1 + 0.015)  # 1.5% stop loss for shorts
                take_profit = entry_price * (1 - 0.03)  # 3% take profit for shorts
                print("Short Entry at", entry_price)
                # Execute trade logic here

# Note: 'some_small_threshold' is a value you set to determine what qualifies as "no shadow."

NameError: name 'compute_heikin_ashi' is not defined